# Выбор локации для скважины

<b>Краткое описание проекта</b><br>
Необходимо определить, локацию для новой скважины для добывающей компании на основании проб нефти в 3 регионах: в каждом 10 000 месторождений, известны данные о качестве нефти и объёме её запасов. 
Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль, а также проанализировать возможную прибыль и риски техникой *Bootstrap.*

<br>
<br>
<b>Цель проекта: </b>Определить регион, где добыча принесет максимальную прибыль и где будет минимальный риск убытков<br><br>
<b>Задачи проекта:</b><br>
<li>Открыть и проанализировать данные. Подгоиовить их к построению модели и анализу</li>
<li>Построить модели линейной регрессии для каждого региона, оценить целевые метрики</li>
<li>Найти точку безубыточности и посчитать прибыль на построенной модели</li>
<li>Провести процедуру бутстрепинга, оценить доверительный интервал и риски убытков</li>
<li>Предложить наилучший регион для разработки</li>


## Загрузка и подготовка данных

Сделаем все необходимые имопрты:

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
RANDOM_STATE = 12345
WELLS_CHECK_COUNT = 500
WELLS_COUNT = 200

Откроем и изучим файлы с даннными:

In [2]:
def open_file_print_info(filepath,heads,fname): #функция для открытия файлов и вывода общей информации
    file = pd.read_csv(filepath)
    print('Информация о файле ',fname)
    print()
    print('Первые ',heads,'строк: ')
    display(file.head(heads))
    print()
    print('Общая информация: ')
    print(file.info())
    #Проверим, есть ли дублирующиеся значения в  датасете:
    print()
    print('Количество дублирующихся строк: ',file.duplicated().sum())
    print('Количество дублирующихся значений в столбце id',len(file) - len(file['id'].unique()))
    print()
    print('Строки с дублирующимися значениями в столбце id:')
    display(file[file['id'].duplicated() == True].sort_values(by = 'id',ascending = True).head(3))
    print('Матрица корреляции: ')
    print(file.drop('id',axis=1).corr())
    return file
geo_data_0 = open_file_print_info('/datasets/geo_data_0.csv',3,'geo_data_0')

Информация о файле  geo_data_0

Первые  3 строк: 


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647



Общая информация: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Количество дублирующихся строк:  0
Количество дублирующихся значений в столбце id 10

Строки с дублирующимися значениями в столбце id:


,id,f0,f1,f2,product
66136,74z30,1.084962,-0.312358,6.990771,127.643327
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453


Матрица корреляции: 
               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000


In [3]:
geo_data_1 = open_file_print_info('/datasets/geo_data_1.csv',3,'geo_data_1')

Информация о файле  geo_data_1

Первые  3 строк: 


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305



Общая информация: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Количество дублирующихся строк:  0
Количество дублирующихся значений в столбце id 4

Строки с дублирующимися значениями в столбце id:


,id,f0,f1,f2,product
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886


Матрица корреляции: 
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000


In [4]:
geo_data_2 = open_file_print_info('/datasets/geo_data_2.csv',3,'geo_data_2')

Информация о файле  geo_data_2

Первые  3 строк: 


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910



Общая информация: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Количество дублирующихся строк:  0
Количество дублирующихся значений в столбце id 4

Строки с дублирующимися значениями в столбце id:


,id,f0,f1,f2,product
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


Матрица корреляции: 
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


Все файлы с данными содержат информацию одинаковой структуры: данные по скважинам. Все данные (за исключеним id) - числовые, типы данных совпадают во всех трех файлах, пропусков нет. Можно видеть, что в каждом регионе есть несколько повторяющихся идентификаторов: вероятнее всего, данные по этим скважинам были внесены в несколько строк (однако, возможно, и ошибка в присвоении идентификаторов). Ввиду того, что совершенно неизвестно, что означают признаки f1, f2, f3, производить какие-либо математические операции над ними не стоит (вероятнее всего, конечно, сумма, но утверждать этого однозначно мы не можем). Поэтому оставим данные как есть.
<br>
можно видеть, что в регионе 1 наблюдается очень высокая корреляция признака f2 с целевым параметром. В остальных регионах корреляция с этим признаком намного ниже (чуть меньше 0.5), также можно видеть,что в регионах 1 и 2 корреляция признаков f0 и f1 отрицтельна по отношению к целевому и низка по модулю, но в регионе 0 корееляция признака f0 с целевым положительная и значительно выше по модулю. Можно точно сказать, что влияние признаков в указанных трех регионах различно. Значит и модели получатся разными. Признак f2 во всех трех регионах самый "весомый".
<br>
Теперь данные готовы к обучению

## Обучение и проверка модели

Разделим выборку на тренировочную и валидационную. Целевой признак для модели - product

In [5]:
def train_test_split_func(geo_data,filename): #функция разделения выборки на обучающую и валидационную
    geo_features = geo_data.drop(['product','id'], axis = 1)
    geo_target = geo_data['product']
    geo_features_train, geo_features_valid, geo_target_train, geo_target_valid = train_test_split(geo_features, geo_target, test_size = 0.25, random_state = RANDOM_STATE)
    print('Результаты разделения выборки на тренировочную и валидационную для датасета: ',filename)
    print('Размер тренировочной выборки: ', geo_features_train.shape[0])
    print('Размер тестовой выборки: ', geo_features_valid.shape[0])
    print('Размер тренировочной выборки по отношению ко всей выборке: ', geo_features_train.shape[0] / geo_features.shape[0])
    print('Размер тестовой выборки по отношению ко всей выборке: ', geo_features_valid.shape[0] / geo_features.shape[0])
    return geo_features_train, geo_features_valid, geo_target_train, geo_target_valid

def make_linear_model(geo_data,filename):
    #разделим выборки на обучающую и валидационную
    geo_features_train, geo_features_valid, geo_target_train, geo_target_valid = train_test_split_func(geo_data,filename)
    #создадим и обучим модель, посчитаем предсказания
    model = LinearRegression()
    model.fit(geo_features_train, geo_target_train)
    geo_prediction = model.predict(geo_features_valid)
    #Посчитаем и выведем метрики
    RMSE = np.sqrt(mean_squared_error(geo_target_valid, geo_prediction))
    print()
    print('Метрики по результатам работы модели')
    print('R2: ',model.score(geo_features_valid, geo_target_valid))
    print('Средний запас сырья на валидационной выборке:', geo_target_valid.mean())
    print('Средний  запас предсказанного сырья:', geo_prediction.mean())
    print('RMSE = ', RMSE)
    geo_prediction = pd.Series(geo_prediction,index = geo_target_valid.index)
    return model, geo_prediction, geo_target_valid

#создадим модель для geo_data_0
geo_linear_model_0, geo_prediction_0, geo_target_valid_0 = make_linear_model(geo_data_0,'geo_data_0')

Результаты разделения выборки на тренировочную и валидационную для датасета:  geo_data_0
Размер тренировочной выборки:  75000
Размер тестовой выборки:  25000
Размер тренировочной выборки по отношению ко всей выборке:  0.75
Размер тестовой выборки по отношению ко всей выборке:  0.25

Метрики по результатам работы модели
R2:  0.27994321524487786
Средний запас сырья на валидационной выборке: 92.07859674082927
Средний  запас предсказанного сырья: 92.59256778438038
RMSE =  37.5794217150813


In [6]:
#создадим модель для geo_data_1
geo_linear_model_1, geo_prediction_1, geo_target_valid_1 = make_linear_model(geo_data_1,'geo_data_1')

Результаты разделения выборки на тренировочную и валидационную для датасета:  geo_data_1
Размер тренировочной выборки:  75000
Размер тестовой выборки:  25000
Размер тренировочной выборки по отношению ко всей выборке:  0.75
Размер тестовой выборки по отношению ко всей выборке:  0.25

Метрики по результатам работы модели
R2:  0.9996233978805126
Средний запас сырья на валидационной выборке: 68.72313602435997
Средний  запас предсказанного сырья: 68.728546895446
RMSE =  0.893099286775616


Обучим модель, посчитаем RMSE и средний заппас предсказанного сырья

In [7]:
#создадим модель для geo_data_2
geo_linear_model_2, geo_prediction_2, geo_target_valid_2 = make_linear_model(geo_data_2,'geo_data_2')

Результаты разделения выборки на тренировочную и валидационную для датасета:  geo_data_2
Размер тренировочной выборки:  75000
Размер тестовой выборки:  25000
Размер тренировочной выборки по отношению ко всей выборке:  0.75
Размер тестовой выборки по отношению ко всей выборке:  0.25

Метрики по результатам работы модели
R2:  0.20524758386040443
Средний запас сырья на валидационной выборке: 94.88423280885438
Средний  запас предсказанного сырья: 94.96504596800489
RMSE =  40.02970873393434


<b>Вывод</b><br>
Средний запас предсказанного сырья практически равен среднему запасу сырья на валидационной выборке, это верно для всех моделей. <br>
Можно видеть, что в регионах geo_data_0 и geo_data_2 довольно высокие (по сравнению с geo_data_1) значения среднего запаса сырья, при этом здесь же и разброс гораздо выше. То есть именно в этих регионах должны быть самые прибыльные шахты. В goe_data_1 средний запас горздо ниже и разброс меньше. <br> 
При этом качество модели гораздо выше и для geo_data_0, и для geo_data_0, чем для geo_data_1. Однако все модели лучше средних.
<br>
Перейдем к расчету прибыли

## Подготовка к расчёту прибыли

Посчитем необходимые для расчета прибыли переменные:

In [8]:
#расчитаем затраты на бурение одной скважины. Известно, что общий бюджет на разработку скважин - 10 млр.руб, 
#а разрабатывают 200 скважин. Тогда затраты на разработку одной скважины (используем константу WELLS_COUNT):
drilling_costs = 10000000000 / WELLS_COUNT
#по условию задачи известно, что доход от тысячи баррелей- 450000 рублей. Объявим в переменной
barrel_income = 450000
#Посчитаем точку безубыточности: минимальный объем в скважине, при котором компания не несет убытки (т.е. прибыль равна нулю). 
min_vol = drilling_costs / barrel_income

#выведем на экран полученные значения:
print('Затраты на разработку одной скважины, руб: ', drilling_costs)
print('Доход от тысячи баррелей, руб: ', barrel_income)
print('Точка безубыточности: минимальный объем, тыс баррелей: ', min_vol)
print('Отношение среднего объема добываемой нефти (по региону geo_data_0) ко значению в точке безубыточности: ', geo_data_0['product'].mean() / min_vol)
print('Отношение среднего объема добываемой нефти (по региону geo_data_1) ко значению в точке безубыточности: ', geo_data_1['product'].mean() / min_vol)
print('Отношение среднего объема добываемой нефти (по региону geo_data_2) ко значению в точке безубыточности: ', geo_data_2['product'].mean() / min_vol)


Затраты на разработку одной скважины, руб:  50000000.0
Доход от тысячи баррелей, руб:  450000
Точка безубыточности: минимальный объем, тыс баррелей:  111.11111111111111
Отношение среднего объема добываемой нефти (по региону geo_data_0) ко значению в точке безубыточности:  0.8325000000000001
Отношение среднего объема добываемой нефти (по региону geo_data_1) ко значению в точке безубыточности:  0.6194250000000001
Отношение среднего объема добываемой нефти (по региону geo_data_2) ко значению в точке безубыточности:  0.8550000000000003


<b>Вывод</b><br>
Как можно видеть, во всех регионах средний объем добываемой нефти ниже значения в точке безубыточности. Особенно низкое значение в регионе geo_data_1, там же и достаточно маленький разброс значений, то есть, вероятнее всего, этот регион убыточный. Самый высокий средний объем добываемой нефти в регионе geo_data_2, в регионе geo_data_0 значения не намного ниже. В обоих регионах достаточно большой разброс значений, то есть, вероятнее всего, в одном из жтих регионов будут максимально прибыльные скважины.<br>
Расчитаем прибыль и убытки

## Расчёт прибыли и рисков 

<h3>Расчет прибыли по выбранным скважинам по предсказаниям моделей</h3>

In [9]:
def income_count_func(predictions, target): #функция для расчета прибыли
    #выбираем шахты с наилучшим предсказанием:
    best_wells = predictions.sort_values(ascending = False)
    target_wells = target[best_wells.index][:WELLS_COUNT]
    income = 0
    damages_count = 0
    for well in target_wells:
        ic = barrel_income * well - drilling_costs #прибыль в отдельной шахте
        income = income + ic #общая прибыль
        if ic < 0:
            damages_count += 1 #считаем общее количество убточных шахт
    return income, damages_count
def print_income(predictions, target,header): #вывод прибыли на экран
    income, damages_count = np.round(income_count_func(predictions, target))
    print('Прибыль для региона ',header)
    print('прибыль = ', income)
    print('Доля убыточных шахт: ', damages_count / WELLS_COUNT)
    return income

In [10]:
#посчитаем прибыль для региона geo_data_0
income_0 = print_income(geo_prediction_0, geo_target_valid_0, 'geo_data_0')

Прибыль для региона  geo_data_0
прибыль =  3320826043.0
Доля убыточных шахт:  0.095


In [11]:
#посчитаем прибыль для региона geo_data_1
income_1 = print_income(geo_prediction_1, geo_target_valid_1, 'geo_data_1')

Прибыль для региона  geo_data_1
прибыль =  2415086697.0
Доля убыточных шахт:  0.0


In [12]:
#посчитаем прибыль для региона geo_data_2
income_2 = print_income(geo_prediction_2, geo_target_valid_2, 'geo_data_2')

Прибыль для региона  geo_data_2
прибыль =  2710349964.0
Доля убыточных шахт:  0.14


<b>Вывод</b><br>
Наибольшее значение прибыли по полученной модели - в регионе geo_data_0. Наименьшее - geo_data_1. Однко в регионах geo_data_0 и geo_data_2 доля убыточных шахт намного выше, чем в регионе geo_data_1.<br>
Рассчитаем риски и и распределение прибыли для каждого региона:

<h3>Расчет рисков и прибыли для каждого региона</h3><br>

Найдем распределение прибыли:

In [13]:
def bootstrap_func(state,target,probs,count): #функция для бутстрепинга
    incomes = []
    damages = []
    losses = 0
    for i in range(count):
        #выделяем в target_valid блоки по 500 шахт (по условию задачи из 500 выбирается 200 лучших)
        target_subs = target.sample(n=WELLS_CHECK_COUNT,replace = True, random_state = state)
        probs_subs = probs[probs.index.isin(target_subs.index)]
        inc, damages_count = income_count_func(probs_subs,target_subs)
        incomes.append(inc)#считаем распределение прибыли
        damages.append(damages_count) #распределение убыточных шахт
        if inc < 0: #считаем убыточные случаи
            losses = losses + 1
    return pd.Series(incomes), losses, pd.Series(damages)

def count_metrics(state,target,probs,count,header):#расчет и вывод показателей
    incomes, losses, damages = bootstrap_func(state,target,probs,count)
    lower = incomes.quantile(0.05)
    print()
    print('\033[1mРасчет прибыли и доверительного интервала для ', header, '\033[0m')
    print('Средняя прибыль ',np.round(incomes.mean()))
    print('5% квантиль ',np.round(lower))
    print('Вероятность убытков: ', losses / count)
    #print('Средняя доля убыточных шахт: ', damages.mean())
    
state = np.random.RandomState(RANDOM_STATE)
repeat_count = 1000

In [14]:
count_metrics(state,geo_target_valid_0,geo_prediction_0,repeat_count,'geo_data_0')


Расчет прибыли и доверительного интервала для  geo_data_0 
Средняя прибыль  396164985.0
5% квантиль  -37181596.0
Вероятность убытков:  0.069


In [15]:
count_metrics(state,geo_target_valid_1,geo_prediction_1,repeat_count,'geo_data_1')


Расчет прибыли и доверительного интервала для  geo_data_1 
Средняя прибыль  461155817.0
5% квантиль  140214879.0
Вероятность убытков:  0.007


In [16]:
count_metrics(state,geo_target_valid_2,geo_prediction_2,repeat_count,'geo_data_0')


Расчет прибыли и доверительного интервала для  geo_data_0 
Средняя прибыль  392950475.0
5% квантиль  -27722051.0
Вероятность убытков:  0.065


<b>Вывод</b><br>
Согласно поставленному условию, вероятность убытков должна быть не более 2,5%. Этому условию соответсвтует только регион geo_data_1. Помимо всего прочего, в данном регоине, по результатам работы процедуры bootstrap самая высокая средняя прибыль, а также только в этом регионе с вероятностью 95% возможна прибыль (свыше 155 млн.руб.), а не убыток, как во всех остальных регионах. <br>
Таким образом, для разработки предлагается регион geo_data_1